In [ ]:
# ================================
# Epistemic UQ via Sobol indices
# ================================

!pip -q install SALib

import numpy as np
from math import sqrt, log, pi
from dataclasses import dataclass
from typing import Tuple
from scipy.stats import norm
from SALib.sample import saltelli
from SALib.analyze import sobol

# ---------- Fixed physical/mission constants ----------
RHO = 1.225       # kg/m^3
G   = 9.81        # m/s^2
M_PAYLOAD = 2.0   # kg
RANGE = 50e3      # m (one-way)
THOVER = 60.0     # s per leg (4 legs total)
OMEGA  = 1110.0   # rad/s (fixed RPM)

SIGMA_MEAN = 0.13
CD0_MEAN   = 0.012

RHO_B_Wh = 158.0      # Wh/kg
ETA_BATT = 0.85
MARGIN   = 0.30
RHO_B    = RHO_B_Wh * 3600.0  # J/kg

DL_MAX = 250.0
BL_MAX = 0.14

# Aleatory COVs (Task 3)
COV_CD0   = 0.20
COV_SIGMA = 0.12

# ---------- Final SORA design ----------
r_final = 0.150
V_final = 77.2
extra_m = 0.328

# ---------- Utilities ----------
@dataclass
class LogNormalParam:
    mu_log: float
    sig_log: float

def logn_from_mean_cov(mean: float, cov: float) -> LogNormalParam:
    sig2 = log(1.0 + cov**2)
    return LogNormalParam(mu_log=np.log(mean) - 0.5*sig2, sig_log=sqrt(sig2))

CD0_LN   = logn_from_mean_cov(CD0_MEAN, COV_CD0)
SIGMA_LN = logn_from_mean_cov(SIGMA_MEAN, COV_SIGMA)

def from_u_to_physical(u: np.ndarray, mu_log: float, sig_log: float) -> float:
    # lognormal: X = exp(mu + sig * u), u~N(0,1)
    return float(np.exp(mu_log + sig_log*u))

@dataclass
class Design:
    r: float
    V: float
    extra_m: float

@dataclass
class State:
    mass: float
    T_per_rotor: float
    A: float
    CT: float
    E_use: float
    P_hover_per_rotor: float

def sizing_model(des: Design, Cd0_nom: float, sigma_nom: float) -> State:
    r, V, em = des.r, des.V, des.extra_m
    m_total = M_PAYLOAD + 2.0
    for _ in range(120):
        W = m_total * G
        A = pi*r*r
        T = W/4.0
        P_hover = (1.0/0.75) * T**1.5 / np.sqrt(2.0*RHO*A)
        mu = V/(OMEGA*r)
        P_cruise = (sigma_nom*Cd0_nom/8.0) * (1.0 + 4.65*mu*mu) * RHO * A * (OMEGA**3) * (r**3)
        E_req = P_hover*4.0*THOVER + 4.0*P_cruise*(RANGE/V)
        usable_frac = max(1e-6, 1.0 - MARGIN - em)
        m_batt = E_req/(usable_frac * ETA_BATT * RHO_B)
        E_use = (1.0 - MARGIN) * ETA_BATT * RHO_B * m_batt
        P_installed = 4.0 * max(P_hover, P_cruise)
        m_motors = 2.506e-4 * P_installed
        m_ESC    = 3.594e-4 * P_installed
        m_rotors = 4.0*(0.7484*r*r - 0.0403*r)
        m_empty_guess = m_motors + m_ESC + m_rotors
        m_total_tmp   = M_PAYLOAD + m_batt + m_empty_guess
        m_frame       = 0.5 + 0.2*m_total_tmp
        m_empty       = m_empty_guess + m_frame
        m_new = M_PAYLOAD + m_batt + m_empty
        if abs(m_new - m_total) < 1e-6:
            m_total = m_new
            break
        m_total = m_new
    W = m_total * G; A = pi*r*r; T = W/4.0
    CT = T/(RHO*A*(OMEGA*r)**2)
    return State(mass=m_total, T_per_rotor=T, A=A, CT=CT, E_use=E_use, P_hover_per_rotor=P_hover)

def limit_states(des: Design, Cd0: float, sigma: float) -> np.ndarray:
    st = sizing_model(des, Cd0_nom=CD0_MEAN, sigma_nom=SIGMA_MEAN)  # objective uses nominal
    # DL/BL
    DL = st.T_per_rotor / st.A
    g1 = DL_MAX - DL
    BL = st.CT / sigma
    g2 = BL_MAX - BL
    # Energy with uncertain Cd0,sigma
    r, V, A = des.r, des.V, st.A
    mu = V/(OMEGA*r)
    P_cruise = (sigma*Cd0/8.0) * (1.0 + 4.65*mu*mu) * RHO * A * (OMEGA**3) * (r**3)
    E_req = st.P_hover_per_rotor*4.0*THOVER + 4.0*P_cruise*(RANGE/V)
    g3 = st.E_use - E_req
    return np.array([g1, g2, g3], float)

# ---------- FORM for energy pf at fixed design with shifted mean Cd0 ----------
def form_energy_pf(des: Design, mean_Cd0: float) -> float:
    # recompute lognormal params with same COV but new mean
    cd_ln = logn_from_mean_cov(mean_Cd0, COV_CD0)
    sig_ln= SIGMA_LN

    def g_of_u(u):
        Cd0   = from_u_to_physical(np.array([u[0]]), cd_ln.mu_log, cd_ln.sig_log)
        sigma = from_u_to_physical(np.array([u[1]]), sig_ln.mu_log, sig_ln.sig_log)
        return float(limit_states(des, Cd0, sigma)[2])

    def grad_g_u(u):
        u = np.array(u, dtype=float)
        Cd0   = from_u_to_physical(np.array([u[0]]), cd_ln.mu_log, cd_ln.sig_log)
        sigma = from_u_to_physical(np.array([u[1]]), sig_ln.mu_log, sig_ln.sig_log)
        rel = 1e-2
        dCd0   = max(1e-12, rel*Cd0)
        dsigma = max(1e-12, rel*sigma)
        g0   = limit_states(des, Cd0, sigma)[2]
        gpC  = limit_states(des, Cd0+dCd0, sigma)[2]
        gmC  = limit_states(des, Cd0-dCd0, sigma)[2]
        dgdC = (gpC-gmC)/(2.0*dCd0)
        gps  = limit_states(des, Cd0, sigma+dsigma)[2]
        gms  = limit_states(des, Cd0, sigma-dsigma)[2]
        dgds = (gps-gms)/(2.0*dsigma)
        dCd0_du1   = cd_ln.sig_log * Cd0
        dsigma_du2 = sig_ln.sig_log * sigma
        return np.array([dgdC*dCd0_du1, dgds*dsigma_du2], float)

    # HL-RF
    u = np.zeros(2)
    for _ in range(60):
        g = g_of_u(u)
        grad = grad_g_u(u)
        ng = np.linalg.norm(grad)
        if ng < 1e-12:
            return 0.0 if g>0 else 1.0
        alpha = grad/ng
        beta  = -g/ng
        u_new = beta*alpha
        if np.linalg.norm(u_new-u) < 1e-5:
            u = u_new; break
        u = u_new
    beta = np.linalg.norm(u)
    return float(norm.cdf(-beta))

# ---------- Blade fatigue model ----------
def blade_pf_weibull(des: Design, m_w: float, m_ref: float = 8.0, p0_at_u1: float = 1e-6) -> float:
    """
    Per-flight fatigue failure probability:
      pf = 1 - exp( -(u/theta)^{m_w} ),  u = (CT/sigma_mean)/BL_MAX  at this design.
    theta calibrated so that at u=1 and m=m_ref, pf = p0_at_u1.
    """
    st = sizing_model(des, CD0_MEAN, SIGMA_MEAN)
    u = (st.CT / SIGMA_MEAN)/BL_MAX
    theta = (-np.log(1.0 - p0_at_u1))**(-1.0/m_ref)
    return float(1.0 - np.exp(- (u/theta)**m_w ))

# ---------- Mission-level pf ----------
def mission_pf(des: Design, b_Cd0: float, m_w: float) -> float:
    pf_energy = form_energy_pf(des, mean_Cd0 = b_Cd0 * CD0_MEAN)
    pf_blade  = blade_pf_weibull(des, m_w=m_w)
    pf_total  = 1.0 - (1.0 - pf_energy) * (1.0 - pf_blade)
    return float(np.clip(pf_total, 0.0, 1.0))

# ---------- Sobol problem definition ----------
problem = {
    "num_vars": 2,
    "names": ["b_Cd0", "m_w"],
    "bounds": [[0.8, 1.2],   # drag mean bias ±20%
               [4.0, 12.0]]  # Weibull shape range
}

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.9/778.9 kB 2.9 MB/s eta 0:00:00


In [11]:
# ---------- Sampling & evaluation ----------
N_base = 2048  # increase for tighter CIs
X = saltelli.sample(problem, N_base, calc_second_order=False)
des = Design(r_final, V_final, extra_m)

Y = np.zeros(X.shape[0])
for i, (b, m) in enumerate(X):
    Y[i] = mission_pf(des, b_Cd0=b, m_w=m)

# ---------- Sobol analysis ----------
Si = sobol.analyze(problem, Y, calc_second_order=False, print_to_console=False)

def show(label, arr):
    return label + ":  " + ", ".join(f"{v:0.3f}" for v in arr)

print("S1  (main effects) :", show("", Si['S1']))
print("ST  (total effects):", show("", Si['ST']))
print("\nParameter names:", problem['names'])
print(f"Mean mission pf over the box ~ {np.mean(Y):.3e}")


/tmp/ipython-input-3270217705.py:3: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5.1 Please use `salib.sample.sobol`
  X = saltelli.sample(problem, N_base, calc_second_order=False)
/tmp/ipython-input-1699028419.py:58: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(np.exp(mu_log + sig_log*u))


S1  (main effects) : :  0.695, 0.305
ST  (total effects): :  0.695, 0.305

Parameter names: ['b_Cd0', 'm_w']
Mean mission pf over the box ~ 1.447e-05


/usr/local/lib/python3.11/dist-packages/SALib/util/__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))
